In [34]:
import requests
from urllib.request import Request, urlopen
import re
from bs4 import BeautifulSoup

class Webscrapper:
    def __init__(self, mode):
        self.url = ""
        self.mode = mode
        self.soup = ""
    def getMode(self):
        return self.mode 

    def set_Mode(self, mode):
        self.mode = mode

    def getURL(self):
        return self.url 

    def set_URL(self, url):
        #structure for values tuple (0:template for url, 1:tuple of values)
        if self.getMode() == "url":
            self.url = url
            self.url = self.url.replace(" ", "%20")
        if self.getMode() =="job":
            self.url = url
            print(self.url)

    def createSoup(self):
        req = Request(self.getURL(), headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        self.soup = BeautifulSoup(webpage, 'html.parser')

    def webScrape(self, jbPost=[], searchQuery=["",""]):
        #URL mode will return a list of  tuples of (jobTitle, jobCompany, JobID)
        cleaner =  re.compile('<.*?>')
        jobPosts=[]
        if self.getMode() =="url":
            position = searchQuery[0]
            location = searchQuery[1] 
            position = position.replace(" ", "-").lower()
            location = location.replace(" ", "-").lower()
            searchString = "{}-jobs-in-{}".format(position,location)
            url = 'https://www.reed.co.uk/jobs/{}'.format(searchString)
            self.set_URL((url))
            self.createSoup()
            cards = self.soup.find_all('article', class_='job-result')
            jobIDs = self.getIds(cards)
            index = 0
            while index < 2:
                c = cards[index]
                tempTitle = self.jobTitle(c)
                title = self.cleanhtml(tempTitle, cleaner)
                tempCompany = self.companyName(c)
                company = self.cleanhtml(tempCompany, cleaner)
                jobPosts.append((title,company,jobIDs[index]))
                index += 1
            return jobPosts
        if self.getMode() == "job":
            jobsList = []
            for j in jbPost:
                position = j[0].replace(" ", "-").lower().strip('\n')
                url = 'https://www.reed.co.uk/jobs/{}/{}'.format(position,j[2])
                self.set_URL(url)
                self.createSoup()
                jobDesc= self.jobDescription(self.soup)
                clean = self.cleanhtml(jobDesc, cleaner)
                jobsList.append((j[0],j[1],clean))
            return jobsList

    def jobTitle(self, card):
        jobTitle = card.find('h3').text
        return jobTitle

    #Get Job Company
    def companyName(self, card):  
        companyName = card.find('div', 'posted-by').find('a').text
        return companyName

    def jobDescription(self, jobSoup):
        desc = jobSoup.find('div','description')
        if desc == None:
            desc = jobSoup.find('div','branded-job--description')
        return desc.span.text


    def cleanhtml(self,raw_html,cleaned):
            cleantext = re.sub(cleaned, ' ' , str(raw_html))
            return cleantext 
        
    def getIds(self, cards):
        x=0
        ids = []
        spans = []
        for c in cards:
            spans.append(c.find_all('span', class_='job-result-anchor'))
            x += 1
        for s in spans:
            ids.append(s[0].get('id')[3:])
        return ids

In [35]:
class Job:
    def __init__(self, values):
        self.title = values[0]
        self.company = values[1]
        self.description = values[2]

    def getTitle(self):
        return self.title

    def getCompany(self):
        return self.company

    def getDescription(self):
        return self.description

In [36]:
class Skills:
    def __init__(self, values):
        self.skills = values[0]
        self.softSkills = values[1]

    def getTitle(self):
        return self.skills

    def getCompany(self):
        return self.softSkills

# Webscrape reed.co.uk

In [38]:
import pandas as pd
ws = Webscrapper("url")
job = input() 
JobPosts= ws.webScrape(searchQuery=(str(job),''))
print(JobPosts)
ws.set_Mode('job')
JobPostings = ws.webScrape(jbPost=JobPosts)
print(JobPostings[0])

French
[('\nFrench Teacher\n', 'Reeson Education', '46697628'), ('\nFrench Translator\n', ' Brite Services', '46412225')]
https://www.reed.co.uk/jobs/french-teacher/46697628
https://www.reed.co.uk/jobs/french-translator/46412225
('\nFrench Teacher\n', 'Reeson Education', ' French Teacher  September start\'Outstanding\' schoolCompetitive salariesOpportunity to teach Spanish tooSuitable for an NQT or experienced French Teacher September start for a French Teacher at an amazing school in South London. You will need to be able to effectively teach French from KS3 to KS5 for this French Teacher post. The school is seeking a French Teacher that is passionate about their subject, willing to work as part of a team and be proactive in all areas of their work. If appointed on a permanent contract, the salary will range from £29915 to £45766, depending on qualifications and experience. The School An \'Outstanding\' school in South West London. Mixed, 11 - 18, non-selective. 90% of students stay i